In [1]:
import boto3

In [4]:
ec2 = boto3.client('ec2')
ec2.describe_instances()

{'Reservations': [],
 'ResponseMetadata': {'RequestId': 'e9d95189-2be8-4b04-b8b3-e514c9509981',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e9d95189-2be8-4b04-b8b3-e514c9509981',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '219',
   'date': 'Thu, 13 Feb 2025 10:47:50 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

ec2 = boto3.client('ec2'):
This creates an EC2 client object using the Boto3 library.
The client() method is used to initialize a connection to a specific AWS service—in this case, EC2.
This client will be used to interact with EC2 resources, such as instances, security groups, etc.

ec2.describe_instances():
This command sends a request to AWS to describe EC2 instances.
The describe_instances() method retrieves information about all EC2 instances in your AWS account.

## Creating SSH Key pair
Creating an SSH key pair is important for securely accessing and managing your EC2 instances (or other servers) remotely, without needing to rely on traditional password-based authentication.

An SSH key pair consists of a public key and a private key. The public key is stored on the EC2 instance, and the private key is kept securely on your local machine.
When you attempt to connect to the instance via SSH, the private key is used to authenticate you. This is far more secure than using passwords

In [5]:
resp = ec2.create_key_pair(KeyName='donut')

In [6]:
# saving these credentials
file = open("creds/sheenu.pem", 'w')
file.write(resp['KeyMaterial'])
file.close()

# Create an Amazon EC2 instances

In [7]:
response = ec2.run_instances(
    ImageId = 'ami-0427090fd1714168b',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='donut',
    BlockDeviceMappings=[            #defining the volume
        {
            "DeviceName": "/dev/xvda",
            'Ebs':{
                'DeleteOnTermination': True,    #it ensures, if we delete the intance the volume will also get deleted
                'VolumeSize': 20
            }
        }
    ]

)

1. ec2.run_instances():
This is the Boto3 method used to launch a new EC2 instance with the given parameters. Here's a breakdown of the parameters being passed to it:

2. Parameters in run_instances():
ImageId = 'ami-0427090fd1714168b':

This specifies the Amazon Machine Image (AMI) to use when launching the EC2 instance. The AMI ID 'ami-0427090fd1714168b' refers to a specific pre-configured image (e.g., an operating system, software, or configuration).
This AMI is used to create the initial environment for the instance.
MinCount=1 and MaxCount=1:

These parameters specify how many instances you want to launch.
MinCount=1 means that at least one instance should be launched.
MaxCount=1 means no more than one instance will be launched (effectively ensuring only 1 instance is created).
InstanceType='t2.micro':

This specifies the type of EC2 instance you want to launch.
t2.micro is a small instance type that is eligible for the AWS Free Tier, meaning it’s relatively low-cost and suitable for lightweight workloads.
KeyName='kgptalkie':

This specifies the name of the SSH key pair you want to associate with the instance. The key pair kgptalkie will be used for SSH access to the instance after it’s launched.

3. Block Device Mappings:
This defines how the EBS (Elastic Block Store) volumes are attached to the EC2 instance. The block device mappings in your example are:

BlockDeviceMappings:
This parameter allows you to specify the block storage devices that should be attached to your instance. In this case, it includes a mapping for the root volume (/dev/xvda).

"DeviceName": "/dev/xvda":

This specifies the device name of the root volume. It’s where the operating system and primary files will be stored.
'Ebs'::

Specifies that an EBS volume is to be attached.

'DeleteOnTermination': True:

This ensures that when the EC2 instance is terminated, the EBS volume attached to it will also be automatically deleted. This prevents orphaned EBS volumes that are no longer needed from incurring unnecessary costs.
'VolumeSize': 20:

This defines the size of the EBS volume in GB. In this case, it will create a 20 GB volume as the root volume for the EC2 instance.

In [8]:
response

{'ReservationId': 'r-0333e49e9cfbe14c9',
 'OwnerId': '872515257072',
 'Groups': [],
 'Instances': [{'Architecture': 'x86_64',
   'BlockDeviceMappings': [],
   'ClientToken': 'e935f245-3549-4e0c-9211-dafc29418652',
   'EbsOptimized': False,
   'EnaSupport': True,
   'Hypervisor': 'xen',
   'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2025, 2, 13, 11, 7, 43, tzinfo=tzutc()),
      'AttachmentId': 'eni-attach-0415f8908a2a4d528',
      'DeleteOnTermination': True,
      'DeviceIndex': 0,
      'Status': 'attaching',
      'NetworkCardIndex': 0},
     'Description': '',
     'Groups': [{'GroupId': 'sg-0ba7e9456f78aebda', 'GroupName': 'default'}],
     'Ipv6Addresses': [],
     'MacAddress': '12:1b:50:5a:e9:6b',
     'NetworkInterfaceId': 'eni-0c429d861f807407c',
     'OwnerId': '872515257072',
     'PrivateDnsName': 'ip-172-31-88-151.ec2.internal',
     'PrivateIpAddress': '172.31.88.151',
     'PrivateIpAddresses': [{'Primary': True,
       'PrivateDnsName': 'ip-17

#### to specify the name of the instance
response = ec2.run_instances(
    ImageId='ami-0427090fd1714168b',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='donut',
    BlockDeviceMappings=[
        {
            "DeviceName": "/dev/xvda",
            'Ebs': {
                'DeleteOnTermination': True,
                'VolumeSize': 20
            }
        }
    ],
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'sheenu'
                }
            ]
        }
    ]
)


## Create Security Group and add rules to it
- Security groups control inbound and outbound traffic of the EC2 instance network interface.
- every EC2 instance must have at least one Security Group associated with it. If no Security Group has been specified during the EC2 instance launch, the default Security Group of the default VPC is associated with the instance.

A Security Group in AWS acts as a virtual firewall that controls incoming and outgoing traffic for EC2 instances. It defines which IP addresses, protocols, and ports can access your instance.

In [9]:
response = ec2.create_security_group(
    GroupName = 'sheenugroup',
    Description = "Security group for testing"
)

In [10]:
security_group_id = response['GroupId']
security_group_id

'sg-012d1562fd562804f'

In this group id we need to provide inbound rule so that our ec2 instance can be accessed from external world

In [11]:
# ip, port, traffic type
response = ec2.authorize_security_group_ingress(
    GroupId = security_group_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        },
        {
            'IpProtocol': 'tcp',
            'FromPort': 80,
            'ToPort': 80,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

as of now we have not connected this policy group with our instance that we have created earlier 
lets do that

In [12]:
response = ec2.describe_instances()

instrance_id = response['Reservations'][0]['Instances'][0]['InstanceId']
instrance_id, security_group_id

('i-0e87c7b4656f306b3', 'sg-012d1562fd562804f')

In [13]:
old_security_group_id = response['Reservations'][0]['Instances'][0]['SecurityGroups'][0]['GroupId']

In [14]:
old_security_group_id

'sg-0ba7e9456f78aebda'

In [15]:
ec2.modify_instance_attribute(InstanceId=instrance_id, Groups=[security_group_id])

{'ResponseMetadata': {'RequestId': '086b0b3a-df67-4714-bb54-37a46b2026dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '086b0b3a-df67-4714-bb54-37a46b2026dc',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Thu, 13 Feb 2025 12:54:44 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [16]:
"""
to connect the old security group
ec2.modify_instance_attribute(InstanceId=instrance_id, Groups=[old_security_group_id])
we can also attach multiple security group like this
ec2.modify_instance_attribute(InstanceId=instrance_id, Groups=[old_security_group_id,security_group_id])
"""

'\nto connect the old security group\nec2.modify_instance_attribute(InstanceId=instrance_id, Groups=[old_security_group_id])\nwe can also attach multiple security group like this\nec2.modify_instance_attribute(InstanceId=instrance_id, Groups=[old_security_group_id,security_group_id])\n'

## Start, Stop and Delete Amazon EC2 instances

In [17]:
# status -> running, stopped, terminated, pending etc.

import time

def wait_for_status(instance_id, target_status):
    '''
    This function checks whether an EC2 instance has reached the desired target_status (e.g., "running" or "stopped").
    Loops infinitely (while True) until the instance reaches the target status.
    Calls describe_instances to fetch instance details.
    Extracts the current state of the instance.
           Possible states: "pending", "running", "stopping", "stopped", "shutting-down", "terminated"
    Checks if the current status matches target_status:
    If yes, prints "Instance is in {target_status} state" and exits the loop.
    If no, waits 10 seconds before checking again.
    
    '''
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)


def start_instances(instance_id):
    '''
    This function starts an EC2 instance and waits until it becomes running.
    '''
    print("EC2 Instance Start")
    ec2.start_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'running')

start_instances([instrance_id])


EC2 Instance Start
Instance is in running state


In [18]:
def stop_instances(instance_id):
    '''
    This function is designed to stop an AWS EC2 instance and wait until it reaches the "stopped" state.
    '''
    print("EC2 Instance Stop")
    ec2.stop_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'stopped')

stop_instances([instrance_id])

EC2 Instance Stop
Instance is in stopped state


In [19]:
def terminate_instances(instance_id):
    '''
    This function terminates an AWS EC2 instance and waits until it reaches the "terminated" state.
    '''
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'terminated')

terminate_instances([instrance_id])

EC2 Instance Termination
Instance is in terminated state
